In [1]:
# Import all the necessary libraries
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import ast

# Do not print warnings
warnings.filterwarnings('ignore')

##### Read the necessary datasets

In [2]:
causal_prior = pd.read_csv('data/CausalPriorFiltered.csv')
gene_status = pd.read_csv('data/OncoKBCancerGeneList.csv')
site_effects = pd.read_csv('data/SiteEffects.csv')

In [3]:
def getGeneType(geneName):
    ''' This function returns -1 -> Tumor Supressor Gene
                                1 -> Oncogene
                                0 -> For Both
                                -2 -> For Not Both
                                -3 -> For Unknown
    '''
    DownGeneLabel = pd.read_csv('data/OncoKBCancerGeneList.csv')
    a = DownGeneLabel.loc[(DownGeneLabel['HugoSymbol']) == (geneName)]

    if(a.empty):
        return -3

    isOncogene = (a['IsOncogene']).iloc[0]
    isTumorSupressor = (a['IsTumorSuppressorGene']).iloc[0]

    if(isOncogene == 'No' and isTumorSupressor == 'Yes'):
        return -1
    elif(isOncogene == 'Yes' and isTumorSupressor == 'No'):
        return 1
    elif(isOncogene == 'Yes' and isTumorSupressor == 'Yes'):
        return 0
    elif(isOncogene == 'No' and isTumorSupressor == 'No'):
        return -2


def upstream_gene_status(geneName):
    # Return if gene is neither Oncogene or Tumor Supressor
    geneType = getGeneType(geneName)
    if(geneType == 0 or geneType == -2 or geneType == -3):
        print('The gene is neither Oncogene or Tumor Supressor')
        return

    # Get upstream genes
    up_genes = causal_prior[causal_prior['Output'] == geneName]
    up_genes.reset_index(drop=True, inplace=True)

    # Decode InteractionType to 1 and -1
    up_genes['InteractCode'] = 0
    up_genes['InteractCode'] = up_genes['InteractionType'].apply(lambda x: 1 if str(x) == 'phosphorylates' or str(x) == 'acetylates' or str(x) == 'methylates' else -1)

    # Construct a new column i.e. 'Sites' splitting sites of each gene
    up_genes['Sites'] = ''
    up_genes['Sites'] = up_genes['Location'].str.split(';')

    # Extract site information of that Gene geneName
    gene_sites = site_effects[site_effects['HugoSymbol'] == geneName]
    gene_sites.reset_index(drop=True, inplace=True)

    # Take the Activating or Inhibiting status(1, -1) from gene_sites and put into up_genes['Sites']
    up_genes['SiteStatus'] = ''
    sites = {}
    for index, row in gene_sites.iterrows():
        sites[row['Location']] = row['Status']

    for index, row in up_genes.iterrows():
        temp  = []
        for site in row['Sites']:
            if(site in sites):
                temp.append(sites[site])
            else:
                temp.append(0)
        up_genes.at[index, 'SiteStatus'] = temp

    # Add another column for Tumor Suppressor or Oncogene label
    up_genes['DownGeneLabel'] = geneType

    # Now finally doing computation
    up_genes['Result'] = ''
    for index, row in up_genes.iterrows():
        temp  = []
        for site in row['SiteStatus']:
            result = int(site) * int(row['InteractCode']) * int(row['DownGeneLabel'])
            temp.append(result)

        up_genes.at[index, 'Result'] = temp

    # Labal genes type
    up_genes['RootGeneLabel'] = ''
    for index, row in up_genes.iterrows():
        up_genes.at[index, 'RootGeneLabel'] = getGeneType(row['Gene'])

    return up_genes

In [4]:
def get_information_from_down_gene(gene):
    # Lets get all the downstream gene of gene
    down_nodes = causal_prior[causal_prior['Gene'] == gene]
    down_nodes.reset_index(drop=True, inplace=True)
    
    # filter genes which is listed in OncoKBCancerGeneList
    col1 = []
    col2 = []
    for i in down_nodes['Output']:
        if(getGeneType(i) == 1 or getGeneType(i) == -1):
            col1.append(i)
            col2.append(getGeneType(i))
    down_known_gene = pd.DataFrame(list(zip(col1, col2)), columns=['Gene', 'Type'])
    
    # final step is to get status from genes
    final_result = pd.DataFrame()
    for i in down_known_gene['Gene']:
        output = upstream_gene_status(i)
        result = output.loc[output['Gene'] == gene]
        final_result = final_result.append(result, ignore_index=True)

    return final_result

In [5]:
get_information_from_down_gene("RB1")

,Gene,InteractionType,Output,Location,InteractCode,Sites,SiteStatus,DownGeneLabel,Result,RootGeneLabel
0,RB1,phosphorylates,MDM2,S395,1,[S395],[-1],1,[-1],-1
1,RB1,dephosphorylates,MDM2,S269;S260,-1,"[S269, S260]","[0, 1]",1,"[0, -1]",-1
2,RB1,dephosphorylates,ARID4A,S1007;S864,-1,"[S1007, S864]","[-1, -1]",-1,"[-1, -1]",-1
3,RB1,phosphorylates,MDM2,S395,1,[S395],[-1],1,[-1],-1
4,RB1,dephosphorylates,MDM2,S269;S260,-1,"[S269, S260]","[0, 1]",1,"[0, -1]",-1


In [7]:
output = pd.DataFrame()
genes = causal_prior['Gene'].unique()
for gene in genes:
    a = get_information_from_down_gene(gene)
    output = output.append(a, ignore_index=True)
output

,Gene,InteractionType,Output,Location,InteractCode,Sites,SiteStatus,DownGeneLabel,Result,RootGeneLabel
0,RAF1,phosphorylates,RB1,S608,1,[S608],[0],-1,[0],1
1,RAF1,phosphorylates,MYC,T8,1,[T8],[0],1,[0],1
2,RAF1,phosphorylates,HRAS,Y137,1,[Y137],[0],1,[0],1
3,RAF1,phosphorylates,MAP2K2,S226;S222,1,"[S226, S222]","[1, 1]",1,"[1, 1]",1
4,RAF1,phosphorylates,MAP2K1,S218;T292;S298;S222;T286;T386,1,"[S218, T292, S298, S222, T286, T386]","[1, -1, 1, 1, -1, 0]",1,"[1, -1, 1, 1, -1, 0]",1
...,...,...,...,...,...,...,...,...,...,...
5984,ELF1,dephosphorylates,RB1,T821;T826,-1,"[T821, T826]","[1, -1]",-1,"[1, -1]",-3
5985,FOXM1,acetylates,TP53,K373,1,[K373],[-1],-1,[1],-3
5986,KAT6A,acetylates,TP53,K382;K120,1,"[K382, K120]","[-1, -1]",-1,"[1, 1]",-2
5987,DNMT3A,phosphorylates,MECOM,S436,1,[S436],[0],1,[0],-1


In [8]:
output.to_csv('data/outputs/final/final_result.csv', index=False)

In [9]:
get_information_from_down_gene('RAF1')

,Gene,InteractionType,Output,Location,InteractCode,Sites,SiteStatus,DownGeneLabel,Result,RootGeneLabel
0,RAF1,phosphorylates,RB1,S608,1,[S608],[0],-1,[0],1
1,RAF1,phosphorylates,MYC,T8,1,[T8],[0],1,[0],1
2,RAF1,phosphorylates,HRAS,Y137,1,[Y137],[0],1,[0],1
3,RAF1,phosphorylates,MAP2K2,S226;S222,1,"[S226, S222]","[1, 1]",1,"[1, 1]",1
4,RAF1,phosphorylates,MAP2K1,S218;T292;S298;S222;T286;T386,1,"[S218, T292, S298, S222, T286, T386]","[1, -1, 1, 1, -1, 0]",1,"[1, -1, 1, 1, -1, 0]",1
5,RAF1,phosphorylates,BCL2L11,S602;T599,1,"[S602, T599]","[0, 0]",-1,"[0, 0]",1
6,RAF1,phosphorylates,MAPK1,T182;S29;T185;Y184;Y187,1,"[T182, S29, T185, Y184, Y187]","[0, 1, 1, 0, 1]",1,"[0, 1, 1, 0, 1]",1
7,RAF1,phosphorylates,MAPK3,Y204;Y203;T202;T201,1,"[Y204, Y203, T202, T201]","[1, 0, 1, 0]",1,"[1, 0, 1, 0]",1
8,RAF1,dephosphorylates,BRAF,T753;T401;S750;S151,-1,"[T753, T401, S750, S151]","[-1, -1, -1, -1]",1,"[1, 1, 1, 1]",1
9,RAF1,dephosphorylates,KRAS,S181,-1,[S181],[1],1,[-1],1


##### Doing varios analysis on the resultant dataset

In [6]:
output = pd.read_csv('data/outputs/final/final_result.csv')

output['LabelStat'] = ''
output['Predicted'] = ''
for index, row in output.iterrows():
    d = {1: 0, -1: 0, 0: 0}
    for item in ast.literal_eval(row['Result']):
        if(int(item) == 1):
            d[1] += 1
        if(int(item) == -1):
            d[-1] += 1
        if(int(item) == 0):
            d[0] += 1
    output.at[index, 'LabelStat'] = d
    output.at[index, 'Predicted'] = max(d, key=d.get)

output

,Gene,InteractionType,Output,Location,InteractCode,Sites,SiteStatus,DownGeneLabel,Result,RootGeneLabel,LabelStat,Predicted
0,RAF1,phosphorylates,RB1,S608,1,['S608'],[0],-1,[0],1,"{1: 0, -1: 0, 0: 1}",0
1,RAF1,phosphorylates,MYC,T8,1,['T8'],[0],1,[0],1,"{1: 0, -1: 0, 0: 1}",0
2,RAF1,phosphorylates,HRAS,Y137,1,['Y137'],[0],1,[0],1,"{1: 0, -1: 0, 0: 1}",0
3,RAF1,phosphorylates,MAP2K2,S226;S222,1,"['S226', 'S222']","[1, 1]",1,"[1, 1]",1,"{1: 2, -1: 0, 0: 0}",1
4,RAF1,phosphorylates,MAP2K1,S218;T292;S298;S222;T286;T386,1,"['S218', 'T292', 'S298', 'S222', 'T286', 'T386']","[1, -1, 1, 1, -1, 0]",1,"[1, -1, 1, 1, -1, 0]",1,"{1: 3, -1: 2, 0: 1}",1
...,...,...,...,...,...,...,...,...,...,...,...,...
5984,ELF1,dephosphorylates,RB1,T821;T826,-1,"['T821', 'T826']","[1, -1]",-1,"[1, -1]",-3,"{1: 1, -1: 1, 0: 0}",1
5985,FOXM1,acetylates,TP53,K373,1,['K373'],[-1],-1,[1],-3,"{1: 1, -1: 0, 0: 0}",1
5986,KAT6A,acetylates,TP53,K382;K120,1,"['K382', 'K120']","[-1, -1]",-1,"[1, 1]",-2,"{1: 2, -1: 0, 0: 0}",1
5987,DNMT3A,phosphorylates,MECOM,S436,1,['S436'],[0],1,[0],-1,"{1: 0, -1: 0, 0: 1}",0


In [29]:
output[output['Gene'] == 'RAF1']

,Gene,InteractionType,Output,Location,InteractCode,Sites,SiteStatus,DownGeneLabel,Result,RootGeneLabel,LabelStat,Predicted
0,RAF1,phosphorylates,RB1,S608,1,['S608'],[0],-1,[0],1,"{1: 0, -1: 0, 0: 1}",0
1,RAF1,phosphorylates,MYC,T8,1,['T8'],[0],1,[0],1,"{1: 0, -1: 0, 0: 1}",0
2,RAF1,phosphorylates,HRAS,Y137,1,['Y137'],[0],1,[0],1,"{1: 0, -1: 0, 0: 1}",0
3,RAF1,phosphorylates,MAP2K2,S226;S222,1,"['S226', 'S222']","[1, 1]",1,"[1, 1]",1,"{1: 2, -1: 0, 0: 0}",1
4,RAF1,phosphorylates,MAP2K1,S218;T292;S298;S222;T286;T386,1,"['S218', 'T292', 'S298', 'S222', 'T286', 'T386']","[1, -1, 1, 1, -1, 0]",1,"[1, -1, 1, 1, -1, 0]",1,"{1: 3, -1: 2, 0: 1}",1
5,RAF1,phosphorylates,BCL2L11,S602;T599,1,"['S602', 'T599']","[0, 0]",-1,"[0, 0]",1,"{1: 0, -1: 0, 0: 2}",0
6,RAF1,phosphorylates,MAPK1,T182;S29;T185;Y184;Y187,1,"['T182', 'S29', 'T185', 'Y184', 'Y187']","[0, 1, 1, 0, 1]",1,"[0, 1, 1, 0, 1]",1,"{1: 3, -1: 0, 0: 2}",1
7,RAF1,phosphorylates,MAPK3,Y204;Y203;T202;T201,1,"['Y204', 'Y203', 'T202', 'T201']","[1, 0, 1, 0]",1,"[1, 0, 1, 0]",1,"{1: 2, -1: 0, 0: 2}",1
8,RAF1,dephosphorylates,BRAF,T753;T401;S750;S151,-1,"['T753', 'T401', 'S750', 'S151']","[-1, -1, -1, -1]",1,"[1, 1, 1, 1]",1,"{1: 4, -1: 0, 0: 0}",1
9,RAF1,dephosphorylates,KRAS,S181,-1,['S181'],[1],1,[-1],1,"{1: 0, -1: 1, 0: 0}",-1


In [9]:
summary = pd.DataFrame(columns=['Gene', 'OncoPredictsGenes', 'TMPredictsGenes', 'BothCount', 'ActualLabel'])

for gene in output['Gene'].unique():
    d = {1: 0, -1: 0, 0: 0}
    gene_list = output[output['Gene'] == gene]
    
    for index, row in gene_list.iterrows():
        if(row['Predicted'] == 1):
            d[1] += 1
        if(row['Predicted'] == -1):
            d[-1] += 1
        if(row['Predicted'] == 0):
            d[0] += 1
    
    temp = pd.DataFrame(data={'Gene':[row['Gene']], 'OncoPredictsGenes':[d[1]], 'TMPredictsGenes':[d[-1]], 'BothCount':[d[0]], 'ActualLabel':[getGeneType(row['Gene'])]})
    summary = summary.append(temp, ignore_index=True)

summary

,Gene,OncoPredictsGenes,TMPredictsGenes,BothCount,ActualLabel
0,RAF1,6,1,4,1
1,TEAD1,2,0,2,-3
2,ATM,3,16,14,-1
3,OXSR1,0,0,1,-3
4,RBPJ,0,0,1,-3
...,...,...,...,...,...
1495,ELF1,1,0,0,-3
1496,FOXM1,1,0,0,-3
1497,KAT6A,1,0,0,-2
1498,DNMT3A,0,0,1,-1


In [10]:
summary.to_csv('data/outputs/final/summarized_genes.csv', index=False)

#### Calculate the accuracy

In [15]:
filter_summary = summary[summary['ActualLabel'] != -3]
filter_summary = filter_summary[filter_summary['ActualLabel'] != -2]
filter_summary = filter_summary[filter_summary['ActualLabel'] != 0]
filter_summary.reset_index(drop=True, inplace=True)
filter_summary

,Gene,OncoPredictsGenes,TMPredictsGenes,BothCount,ActualLabel
0,RAF1,6,1,4,1
1,ATM,3,16,14,-1
2,ATR,0,13,9,-1
3,SGK1,6,6,0,1
4,GNB1,7,0,1,1
...,...,...,...,...,...
213,SMG1,0,1,0,-1
214,RAC2,1,0,1,1
215,SMAD2,1,1,0,-1
216,DUSP4,0,2,0,-1


In [16]:
total_count = filter_summary.shape[0]
correct_count = 0
wrong_count = 0
for index, row in filter_summary.iterrows():
    d = {1: row['OncoPredictsGenes'], -1: row['TMPredictsGenes']}
    if(int(row['ActualLabel']) == max(d, key=d.get)):
        correct_count += 1
    else:
        wrong_count += 1

print(total_count)
print(correct_count)
print(wrong_count)

218
166
52


#### Accuracy percentage


In [17]:
accuracy = (correct_count / total_count) * 100
accuracy

76.14678899082568